In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.naive_bayes import MultinomialNB as MNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data = pd.read_csv('/content/drive/My Drive/ML_Assignment4/yelp_labelled.txt', sep='\t', header=None)

In [ ]:
data

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [ ]:
X = data.iloc[:,0].values
y = data.iloc[:,-1].values

In [ ]:
print(X)

In [ ]:
new_X = []
for j in range(X.size):
  str1 = nltk.wordpunct_tokenize(X[j])
  i = 0
  while (i<len(str1)):
    if (ord(str1[i][0])<65):
      str1.remove(str1[i])
    elif (ord(str1[i][0])>90 and ord(str1[i][0])<97):
      str1.remove(str1[i])
    elif (ord(str1[i][0])>122):
      str1.remove(str1[i])
    i+=1
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  str1 = [ps.stem(word) for word in str1 if not word in set(all_stopwords)]
  str1 = ' '.join(str1)
  str1 = str1.lower()
  new_X.append(str1) 

In [ ]:
X = np.array(new_X)

In [ ]:
print(X)

In [ ]:
print(np.unique(y))

[0 1]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, stratify=y)

In [ ]:
X_train.shape

(700,)

In [ ]:
X_train[3]

'i high hope place sinc burger cook charcoal grill unfortun tast fell flat way flat'

In [ ]:
train_dict = {'the':0}
indx = 0
for d in X_train:
  d = d.split(' ')
  for w in d:
    try:
      if (train_dict[w] == -1):
        indx+=1
    except:
      train_dict[w] = indx
      indx+=1

In [ ]:
print(train_dict)

{'the': 0, 'i': 0, 'love': 1, 'fri': 2, 'bean': 3, 'recommend': 4, 'unless': 5, 'car': 6, 'break': 7, 'front': 8, 'starv': 9, 'it': 10, 'busi': 11, 'know': 12, 'all': 13, 'excel': 14, 'restaur': 15, 'highlight': 16, 'great': 17, 'servic': 18, 'uniqu': 19, 'menu': 20, 'beauti': 21, 'set': 22, 'extrem': 23, 'tasti': 24, 'we': 25, 'wait': 26, 'thirti': 27, 'minut': 28, 'seat': 29, 'although': 30, 'vacant': 31, 'tabl': 32, 'folk': 33, 'salad': 34, 'wing': 35, 'ice': 36, 'cream': 37, 'dessert': 38, 'left': 39, 'feel': 40, 'quit': 41, 'satisfi': 42, 'shower': 43, 'area': 44, 'outsid': 45, 'rins': 46, 'not': 47, 'take': 48, 'full': 49, 'mind': 50, 'nude': 51, 'everyon': 52, 'see': 53, 'go': 54, 'back': 55, 'burritto': 56, 'blah': 57, 'veri': 58, 'good': 59, 'though': 60, 'stop': 61, 'place': 62, 'madison': 63, 'ironman': 64, 'friendli': 65, 'kind': 66, 'staff': 67, 'what': 68, 'should': 69, 'hilari': 70, 'yummi': 71, 'christma': 72, 'eve': 73, 'dinner': 74, 'rememb': 75, 'biggest': 76, 'fail'

In [ ]:
X_train1 = np.full((X_train.size,1319),0,int)
i = 0
for d in X_train:
  d = d.split(' ')
  for w in d:
    try:
      X_train1[i][train_dict[w]] += 1
    except:
      pass
  i+=1

In [ ]:
X_train1

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0]])

In [ ]:
clf = MNB(alpha=1)
clf.fit(X_train1, y_train)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [ ]:
X_test1 = np.full((X_test.size,1319),0,int)
i = 0
for d in X_test:
  d = d.split(' ')
  for w in d:
    try:
      X_test1[i][train_dict[w]] += 1
    except:
      pass
  i+=1

In [ ]:
y_pred = clf.predict(X_test1)
accuracy_score(y_test, y_pred)

0.8033333333333333

In [ ]:
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
y_pred1 = clf.predict(X_train1)
accuracy_score(y_train, y_pred1)

0.9442857142857143